<div style="font-size: 200%; font-weight: bold; color: maroon;">DataFrames en SparkR</span> <span style="color: green;">[solución]</div></div>
<!-- v. 1.1 - Paulo Villegas, 2017 -->

Este notebook es una prueba con el objeto `SparkDataFrame` de [Spark R](https://spark.apache.org/docs/latest/sparkr.html)

# Inicialización

Carga de la librería y creadión de la `SparkSession`

In [ ]:
library(SparkR)
spark <- sparkR.session( "local[*]" );

# Lectura de datos

Vamos a leer un [CSV](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.DataFrameReader)  para crear un `SparkDataFrame`.

**Modalidad 2**: le pedimos a Spark que deduzca el esquema

In [ ]:
uni2 <- read.df( "../../DATA/situacion-laboral-titulados-2019.csv", 
                "csv", header=TRUE, comment="#", 
                inferSchema=TRUE )

Verificamos la clase del objeto

In [ ]:
class(uni2)

Y las columnas de la tabla

In [ ]:
names(uni2)

Nos traemos las 6 primeras filas

In [ ]:
head(uni2)

Veamos el esquema de la tabla

In [ ]:
printSchema(uni2)

# Ejercicios

## Comparación entre tipos de universidad

<span style="color: red;">Pregunta:</span> *El ratio de empleados por rama temática calculado antes, ¿varía en función del tipo de universidad?. Es decir, para cada rama temática, ¿la diferencia entre universidad pública y privada es siempre similar o cambia mucho?*

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>

<br/>
<div class="alert alert-block alert-info">Vamos a separarlo en varias celdas, para mayor claridad.<br/>
Empezamos seleccionando sólo las columnas que nos interesan.</div>

In [ ]:
uni.tipo.trabajando = select( uni2, "Rama", "Tipo", "Total", "Trabajando")

<div class="alert alert-block alert-info">Ahora agrupamos a la vez por rama temática y por tipo de universidad. Es decir, un <tt style="font-weight: bold;">groupBy</tt> con dos columnas</div>

In [ ]:
uni.g = groupBy(uni.tipo.trabajando, uni.tipo.trabajando$Rama, uni.tipo.trabajando$Tipo)

<div class="alert alert-block alert-info">
Y agregamos las columnas <em>Total</em> y <em>Trabajando</em> sumando todos los valores de cada grupo.</div>

In [ ]:
uni.tot = summarize( uni.g,
                     Total=sum(uni.tipo.trabajando$Total), 
                     Trabajando=sum(uni.tipo.trabajando$Trabajando) )

In [ ]:
# Echamos un vistazo a lo que tenemos
collect(uni.tot)

<div class="alert alert-block alert-info">Añadimos una columna más, <em>frac</em>, como el ratio entre las columnas <em>Trabajando</em> y <em>Total</em>. Es decir, el porcentaje de alumnos que trabajan.</div>

In [ ]:
uni.fraction = mutate( uni.tot, frac=uni.tot$Trabajando/uni.tot$Total)

In [ ]:
collect(uni.fraction)

<div class="alert alert-block alert-info">Ya lo tenemos. Para poder comparar visualmente los valores en las universidades privadas y públicas, representamos gráficamente los valores, usando colores distintos para público y privado.</div>

In [ ]:
library(ggplot2)
library(stringr)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

In [ ]:

qplot( Rama, frac, data=collect(uni.fraction), color=Tipo, size=I(4) ) +
  theme(text=element_text(size = 20)) +
  scale_x_discrete(labels = function(x) str_wrap(x, width = 12))

<div class="alert alert-block alert-info">
El efecto que podemos apreciar es que, si bien el porcentaje de alumnos trabajando es siempre mayor en las universidades privadas, para las ramas de <em>Ciencias de la Salud</em> y <em>Arquitectura e Ingeniería</em> la diferencia es muy pequeña, prácticamente insignificante. Mientras que para las otras ramas es más clara. 
    
En particular, para <em>Artes y humanidades</em> hay diez puntos de diferencia (de 0.75 a 0.85) y para <em>Ciencias</em> algo menor. En cualquier caso es importante tener en cuenta que precisamente apra estas dos ramas la muestra para Universidades pequeñas es muy pequeña, así que puede haber otros efectos ahí.</div>

## Ratios de desempleados por tipo de universidad y sexo

<span style="color: red;">Pregunta:</span> *el ratio de desempleados/total en la universidad pública ¿es muy diferente para hombres y mujeres?* 

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>

<br/>
<div class="alert alert-block alert-info">En este caso lo vamos a calcular usando los <em>pipes</em> de la librería <tt>magrittr</tt>, lo que reduce el número de pasos intermedios</div>

In [ ]:
library("magrittr")

In [ ]:
# Calculamos la agrupación & agregación que queremos
uni.t1 <- uni2 %>% 
   filter( uni2$Tipo == "públicas" ) %>%
   select( "Sexo", "Total", "Desempleo") %>% 
   groupBy("Sexo" ) %>%
   summarize( Total=sum(uni2$Total), Desempleo=sum(uni2$Desempleo) )

# Sobre la agregación nos calculamos el ratio buscado
uni.t2 <- uni.t1 %>% 
   mutate( ratio=uni.t1$Desempleo/uni.t1$Total )

# Veamos el resultado
head(uni.t2)

<div class="alert alert-block alert-info">Podemos ver que el ratio global de desempleados es mayor para mujeres que para hombres. Aunque la diferencia no es muy considerable (un 8.8% frente a un 6.9%), sí es significativa (casi dos puntos)</div>